In [ ]:
import glob
import polars as pl
from utils import get_sample_article_ids
topic = "elections"

def load_json_file_paths_to_df(paths):
    df_list = [
        pl.read_json(file) for file in paths
    ]  # Read each file into a DataFrame
    combined_df = pl.concat(df_list, how="diagonal")  # Concatenate all DataFrames
    return combined_df


dir_path = f"grouped_json_articles/{topic}"

files = glob.glob(f"{dir_path}/*.json")
seen_ids = get_sample_article_ids()
print("total files pre-clean = ", len(files))

files_no_samples = [x for x in files if not any([seen in x for seen in seen_ids])]
print("total files post-clean = ", len(files_no_samples))

df = load_json_file_paths_to_df(files_no_samples)
pl.Config.set_tbl_rows(len(df))

total files pre-clean =  5624
total files post-clean =  5397


In [78]:
left = df.filter(pl.col("ID") == "qZPpxN4pJD3orXkl").select("content").item()
center = df.filter(pl.col("ID") == "5JAXnLSys9Po1z0n").select("content").item()
right = df.filter(pl.col("ID") == "qz7Gy6yC3sryqgXe").select("content").item()

In [87]:
from utils import load_json, convert_text_bias_to_numerical, write_back_file
from llama import generate_bias_groq_with_samples, get_groq_client
import importlib
import utils
importlib.reload(utils) 

investigate = glob.glob(f"bert_generated_text/{topic}/*.json")
client = get_groq_client()
file = investigate[0]

for file in investigate:
    json = load_json(file)
    sample = ("{text: " + left + "\nbias: left}" 
                        + "\n\n{text: " + right + "\nbias: right}"
                        + "\n\n{text: " + center + "\nbias: center}")

    prompt = "{text: " + json["original_text"] + "}"

    try:
        bias = generate_bias_groq_with_samples(prompt, sample, client)
        num_bias = convert_text_bias_to_numerical(bias)

        write_back_file(file, {"llama_bias_with_sample": num_bias}, json)
    except Exception as e:
        print(f"error on file={file},\nerror={e}")
        

error on file=bert_generated_text/elections/generated_20V8kjXbJsKJaAsV.json,
error=Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': "{\n   'bias': 'center'\n}"}}
